In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model

In [3]:
# Load data
data = pd.read_csv('/content/drive/My Drive/M2 GL/PFE/Data/hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing_v3.csv')

In [4]:
export_dir = '/content/drive/My Drive/M2 GL/PFE/new Tajweed rule model/AI models/v2'

In [5]:
abdul_basit = data[data['recitor_en'] == 'Abdul Basit']
yassin_aljazaery = data[data['recitor_en'] == 'Yassin Al Jazaery']
ibrahim_aldosary = data[data['recitor_en'] == 'Ibrahim_Aldosary']
al_husary = data[data['recitor_en'] == 'Al husary']

In [6]:
splitted_data_info_np = np.empty((0, 6))
models_information_np = np.empty((0, 5))

In [7]:
max_Y = {'madd_6_Lazim': 2, 'madd_246': 3, 'madd_6': 6, 'madd_2': 5, 'Ikhfaa': 9, 'Idgham': 13, 'tafkhim': 24, 'qalqala': 6, 'imala': 7}
max_X = 8000

In [11]:
def data_preparation(reciter_data, tajweed_rule):
  # Extract 'mfcc' and tajweed_rule columns as lists of strings
  X_raw = reciter_data['mfcc'].astype(str).tolist()
  Y_raw = reciter_data[tajweed_rule].astype(str).tolist()

  # Preprocess the input data (X)
  X = [tf.constant(eval(x)) for x in X_raw]
  Y = [tf.constant(eval(x)) for x in Y_raw]

  # Pad sequences in Y and X to ensure all have the same length
  Y_padded = tf.keras.preprocessing.sequence.pad_sequences(Y, maxlen=max_Y[tajweed_rule], padding='post', dtype='int32', value=-1)
  X_padded = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_X, padding='post', dtype='float32')

  # Split the data into training and testing sets
  X_train, X_test, Y_train, Y_test = train_test_split(X_padded, Y_padded, test_size=0.2, random_state=10)
  return X_train, X_test, Y_train, Y_test

In [12]:
def tajweed_rule_model(reciter1, reciter2, reciter3, reciter4, tajweed_rule):
  global splitted_data_info_np, models_information_np, data
  # data preparation
  reciter1_X_train, reciter1_X_test, reciter1_Y_train, reciter1_Y_test = data_preparation(reciter1, tajweed_rule)
  reciter2_X_train, reciter2_X_test, reciter2_Y_train, reciter2_Y_test = data_preparation(reciter2, tajweed_rule)
  reciter3_X_train, reciter3_X_test, reciter3_Y_train, reciter3_Y_test = data_preparation(reciter3, tajweed_rule)
  reciter4_X_train, reciter4_X_test, reciter4_Y_train, reciter4_Y_test = data_preparation(reciter4, tajweed_rule)

  # Update splitted_data_info with information about each reciter
  for reciter_X_train, reciter_X_test, reciter_Y_train, reciter_Y_test, reciter_data in [
      (reciter1_X_train, reciter1_X_test, reciter1_Y_train, reciter1_Y_test, reciter1),
      (reciter2_X_train, reciter2_X_test, reciter2_Y_train, reciter2_Y_test, reciter2),
      (reciter3_X_train, reciter3_X_test, reciter3_Y_train, reciter3_Y_test, reciter3),
      (reciter4_X_train, reciter4_X_test, reciter4_Y_train, reciter4_Y_test, reciter4)]:

      splitted_data_info_np = np.append(splitted_data_info_np, [[
              tajweed_rule,
              reciter_data.iloc[0]['recitor_en'],
              len(reciter_X_train),
              len(reciter_X_test),
              len(reciter_Y_train),
              len(reciter_Y_test)
              ]], axis=0)

  # concatenate data
  # training data
  X_train = np.concatenate([reciter1_X_train, reciter2_X_train, reciter3_X_train, reciter4_X_train], axis=0)
  Y_train = np.concatenate([reciter1_Y_train, reciter2_Y_train, reciter3_Y_train, reciter4_Y_train], axis=0)

  # testing data
  X_test = np.concatenate([reciter1_X_test, reciter2_X_test, reciter3_X_test, reciter4_X_test], axis=0)
  Y_test = np.concatenate([reciter1_Y_test, reciter2_Y_test, reciter3_Y_test, reciter4_Y_test], axis=0)

  splitted_data_info_np = np.append(splitted_data_info_np, [[
          tajweed_rule,
          'all reciters',
          len(X_train),
          len(X_test),
          len(Y_train),
          len(Y_test)
          ]], axis=0)

  # Normalize input data by scaling each sequence individually
  scaler = StandardScaler()
  X_train_scaled = np.array([scaler.fit_transform(seq) for seq in X_train])
  X_test_scaled = np.array([scaler.transform(seq) for seq in X_test])

  # Define a simple neural network model
  input_shape = X_train_scaled[0].shape  # Shape of each mfcc sequence
  output_shape = Y_train.shape[1]  # Dimension of output (number of units in output layer)

  input_layer = Input(shape=input_shape)
  flatten_layer = Flatten()(input_layer)  # Flatten the sequence to a 1D vector
  hidden_layer = Dense(64, activation='relu')(flatten_layer)
  output_layer = Dense(output_shape, activation='linear')(hidden_layer)  # Define the output layer with the correct units

  model = Model(inputs=input_layer, outputs=output_layer)

  # Compile the model
  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

  # Train the model
  model.fit(X_train_scaled, Y_train, epochs=50, batch_size=32, validation_split=0.1)

  #export the model
  model_filename = f'{tajweed_rule}_tajweed_rule_model'
  model_path = os.path.join(export_dir, model_filename)
  keras.models.save_model(model, model_path)

  # Make predictions on test data
  predictions = model.predict(X_test_scaled)

  # Evaluate the model with adjusted predictions
  predictions[predictions < 0] = -1
  predictions = np.round(predictions).astype('int32')
  loss, accuracy = model.evaluate(X_test_scaled, predictions)

  print(f"Test Loss: {loss:.4f}, Test accuracy : {accuracy:.4f}")
  models_information_np = np.append(models_information_np, [[
          model_filename,
          "{:.4f}".format(loss),
          "{:.4f}".format(accuracy),
          "{:.2f}".format(accuracy*100),
          model_path]], axis=0)

In [13]:
for rule in max_Y.keys():
  tajweed_rule_model(abdul_basit, yassin_aljazaery, ibrahim_aldosary, al_husary, rule)

Epoch 1/50
27/27 [==============================] - 4s 129ms/step - loss: 153.0115 - accuracy: 0.1659 - val_loss: 6.4930 - val_accuracy: 0.4688
Epoch 2/50
27/27 [==============================] - 2s 85ms/step - loss: 12.0393 - accuracy: 0.0421 - val_loss: 4.4242 - val_accuracy: 0.1354
Epoch 3/50
27/27 [==============================] - 2s 81ms/step - loss: 11.8205 - accuracy: 0.0129 - val_loss: 3.4431 - val_accuracy: 0.0625
Epoch 4/50
27/27 [==============================] - 2s 79ms/step - loss: 11.7290 - accuracy: 0.0105 - val_loss: 3.3928 - val_accuracy: 0.0625
Epoch 5/50
27/27 [==============================] - 2s 87ms/step - loss: 11.7215 - accuracy: 0.0093 - val_loss: 3.3729 - val_accuracy: 0.0625
Epoch 6/50
27/27 [==============================] - 4s 142ms/step - loss: 11.7075 - accuracy: 0.0093 - val_loss: 3.3559 - val_accuracy: 0.0521
Epoch 7/50
27/27 [==============================] - 3s 120ms/step - loss: 11.6924 - accuracy: 0.0093 - val_loss: 3.3426 - val_accuracy: 0.0521
Ep

In [14]:
def print_model_summary(loaded_model, tajweed_rule):
  print(f'******* Tajweed rule {tajweed_rule} model *******')
  loaded_model.summary()
  print('\n')

In [15]:
for rule in max_Y.keys():
    model_filename = f'{rule}_tajweed_rule_model'
    model_path = os.path.join(export_dir, model_filename)

    # Load the saved model
    loaded_model = tf.keras.models.load_model(model_path)

    print_model_summary(loaded_model, rule)

******* Tajweed rule madd_6_Lazim model *******
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8000, 13)]        0         
                                                                 
 flatten (Flatten)           (None, 104000)            0         
                                                                 
 dense (Dense)               (None, 64)                6656064   
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 6656194 (25.39 MB)
Trainable params: 6656194 (25.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


******* Tajweed rule madd_246 model *******
Model: "model_1"
__________________________________________

In [16]:
# how data is splitted
columns1 = ['tajweed_rule', 'data_of', 'X_train_nb_samples', 'X_test_nb_samples', 'Y_train_nb_samples', 'X_test_nb_samples']
splitted_data_info = pd.DataFrame(data=splitted_data_info_np, columns=columns1)

# save models information
columns2 = ['Model', 'Loss', 'Accuracy', 'Accuracy %', 'Path_to_the_model']
models_information = pd.DataFrame(data=models_information_np, columns=columns2)

In [17]:
splitted_data_info

,tajweed_rule,data_of,X_train_nb_samples,X_test_nb_samples,Y_train_nb_samples,X_test_nb_samples
0,madd_6_Lazim,Abdul Basit,238,60,238,60
1,madd_6_Lazim,Yassin Al Jazaery,238,60,238,60
2,madd_6_Lazim,Ibrahim_Aldosary,238,60,238,60
3,madd_6_Lazim,Al husary,238,60,238,60
4,madd_6_Lazim,all reciters,952,240,952,240
5,madd_246,Abdul Basit,238,60,238,60
6,madd_246,Yassin Al Jazaery,238,60,238,60
7,madd_246,Ibrahim_Aldosary,238,60,238,60
8,madd_246,Al husary,238,60,238,60
9,madd_246,all reciters,952,240,952,240


In [18]:
models_information

,Model,Loss,Accuracy,Accuracy %,Path_to_the_model
0,madd_6_Lazim_tajweed_rule_model,134.8659,0.0458,4.58,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
1,madd_246_tajweed_rule_model,351.7010,1.0000,100.00,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
2,madd_6_tajweed_rule_model,506.4740,0.9417,94.17,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
3,madd_2_tajweed_rule_model,528.2846,1.0000,100.00,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
4,Ikhfaa_tajweed_rule_model,386.9288,0.8542,85.42,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
5,Idgham_tajweed_rule_model,421.5266,1.0000,100.00,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
6,tafkhim_tajweed_rule_model,190.9882,0.8625,86.25,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
7,qalqala_tajweed_rule_model,34.8145,0.9958,99.58,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
8,imala_tajweed_rule_model,1437.6967,0.0750,7.50,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
